In [ ]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
run_hash="aa7a0685"
ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/rf_result.ht')
freq_ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj.ht')

In [5]:
freq_ht.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
     

In [6]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float64 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_

In [17]:
ht=ht.annotate(ac_raw=ht.n_alt_alleles)

In [18]:
ht=ht.annotate(ac=ht.n_alt_alleles)

In [19]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float64 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_

In [20]:
ht.write(f'{tmp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/rf_result_ac_added.ht', overwrite=True)

2020-10-12 10:02:49 Hail: INFO: wrote table with 929613 rows in 500 partitions to hdfs://spark-master:9820//ddd-elgh-ukbb/variant_qc/models/aa7a0685/rf_result_ac_added.ht


In [11]:
allele_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_allele_data.ht')

In [13]:
allele_data_ht.show()

locus,alleles,allele_data.nonsplit_alleles,allele_data.has_star,allele_data.variant_type,allele_data.n_alt_alleles,allele_data.allele_type,allele_data.was_mixed,a_index,was_split
locus<GRCh38>,array<str>,array<str>,bool,str,int32,str,bool,int32,bool
chr1:12938,"[""GCAAA"",""G""]","[""GCAAA"",""G""]",false,"""indel""",1,"""del""",false,1,false
chr1:13024,"[""G"",""A""]","[""G"",""A""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13087,"[""A"",""G""]","[""A"",""G""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13116,"[""T"",""C""]","[""T"",""C""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13130,"[""C"",""T""]","[""C"",""T""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13151,"[""G"",""C""]","[""G"",""C""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13164,"[""G"",""A""]","[""G"",""A""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13176,"[""G"",""T""]","[""G"",""T""]",false,"""snv""",1,"""snv""",false,1,false
chr1:13198,"[""C"",""A""]","[""C"",""A""]",false,"""snv""",1,"""snv""",false,1,false


In [15]:
mt=hl.read_matrix_table(f'{tmp_dir}/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj.mt')

In [16]:
mt.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>>
----------------------------------------
Column fields:
    's': str
    'cohort': str
    'known_pop': str
    'gVCF': str
    'group_membership': array<bool>
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoe

In [2]:
ht_bin=hl.read_table('hdfs://spark-master:9820//models/aa7a0685/rf_result_quantile_bins.ht')

In [3]:
ht_bin.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
    'feature_medians_1': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata_1': dict<tuple (
        str
    ), int64> 
    'features_importance_1': dict<str, float64> 
    'features_1': array<str> 
    'test_results_1': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash_1': str 
    'bin_stats': struct {
        adj_singleton_bin: struct {
